In [2]:
import streamlit as st
import pandas as pd
from pathlib import Path
from utils import app_utils
import boto3
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()
aws_access_key_id = os.environ['aws_access_key_id']
aws_secret_access_key = os.environ['aws_secret_access_key']
    
# Creating boto3 session (access the S3 bucket)
s3 = boto3.resource('s3',
						aws_access_key_id = aws_access_key_id, 
						aws_secret_access_key = aws_secret_access_key)

In [14]:
from io import BytesIO
# Function to read CSV file from S3 bucket
def read_csv_from_s3(s3, bucket_name, file_name):
# Get the S3 bucket and object
    bucket = s3.Bucket(bucket_name)
    obj = bucket.Object(file_name)
    xls_data = obj.get()['Body'].read()

    # Create a Pandas DataFrame from the XLS data
    df = pd.read_excel(BytesIO(xls_data), engine='openpyxl')
    # Read CSV data from S3 object
    # df = df[~((df['anho']==2023)&(df['semana_no']==53))]
    return df


In [22]:
data = read_csv_from_s3(s3 = s3, bucket_name = 'sipsatracker',file_name = 'week_1_Anexo_Bol_Semanal_SIPSA_enero_04_2013.xls')

BadZipFile: File is not a zip file

In [12]:
data[data['categoria'].isnull()]['anho'].unique()

array([2018, 2019, 2020, 2021, 2022, 2023], dtype=int64)

In [167]:
import boto3
import pandas as pd
from io import BytesIO

def read_xls_from_s3(bucket_name, file_name):
    # Create an S3 resource
    s3 = boto3.resource('s3')

    try:
        # Retrieve the S3 object
        s3_object = s3.Object(bucket_name, file_name)

        # Read the XLS file from S3
        xls_data = s3_object.get()['Body'].read()

        # Create a Pandas DataFrame from the XLS data
        if file_name.split('.')[-1] == 'xls': 
	        df = pd.read_excel(xls_data)
        if file_name.split('.')[-1] == 'csv': 
	        df = pd.read_csv(s3_object.get()['Body'])
        # df = pd.read_excel(BytesIO(xls_data), engine='xlrd')

        return df

    except Exception as e:
        print(f"Error reading file from S3: {e}")
        return None

# Usage
bucket_name = 'sipsatracker'
# file_name = 'week_1_Anexo_Bol_Semanal_SIPSA_enero_04_2013.xls'
file_name = 'full_report.csv'

df = read_xls_from_s3(bucket_name, file_name)
df


,ciudad,precio_minimo,precio_maximo,precio_medio,tendencia,categoria,producto,mercado,semana_no,anho,region
0,barranquilla,2867,3000,2922,+,verduras_hortalizas,acelga,barranquillita,45,2012,caribe
1,bogota,300,333,303,--,verduras_hortalizas,acelga,corabastos,45,2012,andina
2,bucaramanga,1000,1867,1296,--,verduras_hortalizas,acelga,centroabastos,45,2012,andina
3,cali,1050,1500,1281,+++,verduras_hortalizas,acelga,cavasa,45,2012,pacífico
4,cali,1050,1433,1242,+++,verduras_hortalizas,acelga,santa helena,45,2012,pacífico
...,...,...,...,...,...,...,...,...,...,...,...
2541876,pamplona,1750,1833,1778,=,productos_procesados,vinagre,NaN,9,2023,andina
2541877,pereira,6216,6840,6588,-,productos_procesados,vinagre,mercasa,9,2023,andina
2541878,santa marta,1525,1583,1553,+,productos_procesados,vinagre,NaN,9,2023,caribe
2541879,sincelejo,1333,1875,1542,-,productos_procesados,vinagre,nuevo mercado,9,2023,caribe


In [166]:
df

,Value
0,10.000000
1,15.000000
2,12.000000
3,18.000000
4,114.900000
5,17.000000
6,14.000000
7,20.000000
8,22.000000
9,21.000000


In [177]:
index_weights =  {'verduras_hortalizas':0.094,
                        'frutas_frescas':0.050,
                        'tuberculos_raices_platanos':0.041,
                        'granos_cereales':0.186,
                        'huevos_lacteos':0.214,
                        'carnes':0.237,
                        'pescados':0.030, 
                        'productos_procesados':0.148}
    
year_list, week_list, sipsa_index_list = [],[],[]

    # Filter the DataFrame for the desired year and week
for year in df['anho'].unique():
        
    for week in df.loc[df['anho']==year,'semana_no'].unique():
            
        df_filtered = df[(df['anho'] == year) & (df['semana_no'] == week)]
        # Calculate the weighted sum for each category
        year_list.append(year)
        week_list.append(week)
        sipsa_index_list.append((df_filtered['precio_medio'] * df_filtered['categoria'].map(index_weights)).sum())

df_final = pd.DataFrame({'year':year_list, 
                            'week': week_list, 
                            'sipsa': sipsa_index_list})


7966717.345000001